In [2]:
import pandas as pd
import json
import requests
import numpy as np

C:\Users\junke\AppData\Local\Temp\ipykernel_20348\913430442.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
def fetch_data_from_url(url):
    """
    Fetch JSON data from the given URL.
    """
    response = requests.get(url)
    response.raise_for_status()  # Raise an exception for bad responses
    return response.json()

def normalize_restaurant_data(restaurant_data):
    """
    Normalize the "restaurants" object in JSON data to a DataFrame.
    """
    return pd.json_normalize(restaurant_data, 'restaurants')

def merge_with_country_code(restaurant_df, country_code_df):
    """
    Merge restaurant data with country code DataFrame based on country ID.
    """
    return pd.merge(restaurant_df, country_code_df, 
                    how='left', left_on='restaurant.location.country_id', 
                    right_on='Country Code')

def extract_required_fields(merged_df):
    """
    Extract required fields from the merged DataFrame.
    """
    return merged_df[['restaurant.R.res_id', 'restaurant.name', 'Country', 
                      'restaurant.location.city', 'restaurant.user_rating.votes', 
                      'restaurant.user_rating.aggregate_rating', 'restaurant.cuisines']]

def rename_columns(final_df):
    """
    Rename columns of the DataFrame.
    """
    renamed_columns = {
        'restaurant.R.res_id': 'Restaurant Id',
        'restaurant.name': 'Restaurant Name',
        'restaurant.location.city': 'City',
        'restaurant.user_rating.votes': 'User Rating Votes',
        'restaurant.user_rating.aggregate_rating': 'User Aggregate Rating',
        'restaurant.cuisines': 'Cuisines'
    }
    return final_df.rename(columns=renamed_columns)

def convert_data_types(final_df):
    """
    Convert data types of specific columns in the DataFrame.
    """
    final_df['User Aggregate Rating'] = final_df['User Aggregate Rating'].astype('float64')
    return final_df

def export_to_csv(final_df, file_path):
    """
    Export DataFrame to a CSV file.
    """
    final_df.to_csv(file_path, index=False)

In [4]:
# URL of the JSON data
url = "https://raw.githubusercontent.com/Papagoat/brain-assessment/main/restaurant_data.json"

# Fetch data from the URL
restaurant_data = fetch_data_from_url(url)

# Normalize restaurant data
restaurant_df = normalize_restaurant_data(restaurant_data)

In [5]:
restaurant_df

,restaurant.R.res_id,restaurant.apikey,restaurant.id,restaurant.name,restaurant.url,restaurant.location.address,restaurant.location.locality,restaurant.location.city,restaurant.location.city_id,restaurant.location.latitude,...,restaurant.has_table_booking,restaurant.events_url,restaurant.establishment_types,restaurant.medio_provider,restaurant.order_url,restaurant.order_deeplink,restaurant.book_url,restaurant.user_rating.custom_rating_text,restaurant.user_rating.custom_rating_text_background,restaurant.user_rating.rating_tool_tip
0,18649486,cba15beb4c265876a9828f242b4cf41c,18649486,The Drunken Botanist,https://www.zomato.com/ncr/the-drunken-botanis...,"Unit 1B & 1C, Upper Ground Floor-C, Building 1...","Cyber Hub, DLF Cyber City",Gurgaon,1,28.4936741035,...,0,https://www.zomato.com/ncr/the-drunken-botanis...,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,308322,cba15beb4c265876a9828f242b4cf41c,308322,Hauz Khas Social,https://www.zomato.com/HauzKhasSocial?utm_sour...,"9-A & 12, Hauz Khas Village, New Delhi",Hauz Khas Village,New Delhi,1,28.5542935327,...,1,https://www.zomato.com/HauzKhasSocial/events#t...,[],1,https://www.zomato.com/HauzKhasSocial/order?ut...,,https://www.zomato.com/HauzKhasSocial/book?utm...,NaN,NaN,NaN
2,18856789,cba15beb4c265876a9828f242b4cf41c,18856789,AIR- An Ivory Region,https://www.zomato.com/ncr/air-an-ivory-region...,"1/83, Third Floor, Club Road, West Punjabi Bag...",Punjabi Bagh,New Delhi,1,28.5478090000,...,1,https://www.zomato.com/ncr/air-an-ivory-region...,[],,NaN,NaN,https://www.zomato.com/ncr/air-an-ivory-region...,NaN,NaN,NaN
3,307374,cba15beb4c265876a9828f242b4cf41c,307374,AMA Cafe,https://www.zomato.com/ncr/ama-cafe-majnu-ka-t...,"House 6, New Colony, Majnu ka Tila, New Delhi",Majnu ka Tila,New Delhi,1,28.7025817618,...,0,https://www.zomato.com/ncr/ama-cafe-majnu-ka-t...,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,18238278,cba15beb4c265876a9828f242b4cf41c,18238278,Tamasha,https://www.zomato.com/ncr/tamasha-connaught-p...,"28, Block A, Kasturba Gandhi Marg, Connaught P...",Connaught Place,New Delhi,1,28.6296624581,...,0,https://www.zomato.com/ncr/tamasha-connaught-p...,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1295,7900186,cba15beb4c265876a9828f242b4cf41c,7900186,Mexico,https://www.zomato.com/hamilton/mexico-hamilto...,"254 Victoria Street, Hamilton Central, Hamilton",Hamilton Central,Hamilton,79,-37.7882501653,...,0,https://www.zomato.com/hamilton/mexico-hamilto...,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN
1296,7900182,cba15beb4c265876a9828f242b4cf41c,7900182,Thai House Express,https://www.zomato.com/hamilton/thai-house-exp...,"303 Victoria Street, Hamilton Central, Hamilton",Hamilton Central,Hamilton,79,-37.7879942113,...,0,https://www.zomato.com/hamilton/thai-house-exp...,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN
1297,7900023,cba15beb4c265876a9828f242b4cf41c,7900023,Indian Star,https://www.zomato.com/hamilton/indian-star-ha...,"20 Alma Street, Hamilton Central, Hamilton",Hamilton Central,Hamilton,79,-37.7855080000,...,0,https://www.zomato.com/hamilton/indian-star-ha...,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN
1298,7900490,cba15beb4c265876a9828f242b4cf41c,7900490,Zealong Tea Estate,https://www.zomato.com/hamilton/zealong-tea-es...,"495 Gordonton Road, Hamilton",Huntington,Hamilton,79,-37.7076580000,...,0,https://www.zomato.com/hamilton/zealong-tea-es...,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# Unpack the lists in the events column vertically
expended_events_df = restaurant_df.explode("restaurant.zomato_events")

# Unpack the dictionaries within each element of the column
unpacked_events_df = pd.json_normalize(expended_events_df["restaurant.zomato_events"])

# Merge the unpacked DataFrame with the original DataFrame
events_df = pd.concat([restaurant_df, unpacked_events_df], axis=1)

In [15]:
# Change event start and end date to a date data type
events_df['event.start_date'] = pd.to_datetime(events_df["event.start_date"])
events_df['event.end_date'] = pd.to_datetime(events_df["event.end_date"])

In [18]:
# Filter rows where either the start or end date falls within the desired month and year
April2019_events_df = events_df[((events_df['event.start_date'].dt.year <= 2019) & (events_df['event.start_date'].dt.month <= 4)) |
                 ((events_df['event.end_date'].dt.year >= 2019) & (events_df['event.end_date'].dt.month >= 4))]

In [30]:
# Extract required Columns
April2019_events_df = April2019_events_df[["event.event_id", "restaurant.id",
                      "restaurant.name", "restaurant.photos_url",
                      "event.title", "event.start_date",
                      "event.end_date"]]

# Rename required columns
columns_to_rename = {
    "event.event_id":"Event Id", 
    "restaurant.id":"Restaurant Id",
    "restaurant.name":"Restaurant Name", 
    "restaurant.photos_url":"Photo URL",
    "event.title":"Event Title", 
    "event.start_date":"Event Start Date",
    "event.end_date":"Event End Date"
}
renamed_events_df = April2019_events_df.rename(columns=columns_to_rename)

renamed_events_df.isna().sum()

Event Id            0
Restaurant Id       0
Restaurant Name     0
Photo URL           0
Event Title         0
Event Start Date    0
Event End Date      0
dtype: int64

In [19]:
April2019_events_df

,restaurant.R.res_id,restaurant.apikey,restaurant.id,restaurant.name,restaurant.url,restaurant.location.address,restaurant.location.locality,restaurant.location.city,restaurant.location.city_id,restaurant.location.latitude,restaurant.location.longitude,restaurant.location.zipcode,restaurant.location.country_id,restaurant.location.locality_verbose,restaurant.switch_to_order_menu,restaurant.cuisines,restaurant.average_cost_for_two,restaurant.price_range,restaurant.currency,restaurant.offers,restaurant.zomato_events,restaurant.opentable_support,restaurant.is_zomato_book_res,restaurant.mezzo_provider,restaurant.is_book_form_web_view,restaurant.book_form_web_view_url,restaurant.book_again_url,restaurant.thumb,restaurant.user_rating.aggregate_rating,restaurant.user_rating.rating_text,restaurant.user_rating.rating_color,restaurant.user_rating.votes,restaurant.user_rating.has_fake_reviews,restaurant.photos_url,restaurant.menu_url,restaurant.featured_image,restaurant.has_online_delivery,restaurant.is_delivering_now,restaurant.has_fake_reviews,restaurant.include_bogo_offers,restaurant.deeplink,restaurant.is_table_reservation_supported,restaurant.has_table_booking,restaurant.events_url,restaurant.establishment_types,restaurant.medio_provider,restaurant.order_url,restaurant.order_deeplink,restaurant.book_url,restaurant.user_rating.custom_rating_text,restaurant.user_rating.custom_rating_text_background,restaurant.user_rating.rating_tool_tip,event.event_id,event.friendly_start_date,event.friendly_end_date,event.friendly_timing_str,event.start_date,event.end_date,event.end_time,event.start_time,event.is_active,event.date_added,event.photos,event.restaurants,event.is_valid,event.share_url,event.show_share_url,event.title,event.description,event.display_time,event.display_date,event.is_end_time_set,event.disclaimer,event.event_category,event.event_category_name,event.book_link,event.types,event.share_data.should_show,event.is_zomato_event,event.url
0,18649486.0,cba15beb4c265876a9828f242b4cf41c,18649486,The Drunken Botanist,https://www.zomato.com/ncr/the-drunken-botanis...,"Unit 1B & 1C, Upper Ground Floor-C, Building 1...","Cyber Hub, DLF Cyber City",Gurgaon,1.0,28.4936741035,77.0883342996,,1.0,"Cyber Hub, DLF Cyber City, Gurgaon",0.0,"Continental, Italian, North Indian, Chinese",1500.0,3.0,Rs.,[],"[{'event': {'event_id': 322331, 'friendly_star...",0.0,0.0,OTHER,0.0,,,https://b.zmtcdn.com/data/pictures/6/18649486/...,4.4,Very Good,5BA829,4765,0.0,https://www.zomato.com/ncr/the-drunken-botanis...,https://www.zomato.com/ncr/the-drunken-botanis...,https://b.zmtcdn.com/data/pictures/6/18649486/...,0.0,0.0,0.0,True,zomato://restaurant/18649486,0.0,0.0,https://www.zomato.com/ncr/the-drunken-botanis...,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,322331.0,06 March,28 August,"Wednesday, 6th March - Wednesday, 28th August",2019-03-06,2019-08-28,23:59:59,20:00:00,1.0,2019-03-06 11:41:21,[{'photo': {'url': 'https://b.zmtcdn.com/data/...,[],1.0,http://www.zoma.to/r/0,0.0,BackToBasic Wednesdays !!\n\n\n,https://www.facebook.com/events/34747730608584...,08:00 pm onwards,06 March - 28 August,0.0,Restaurants are solely responsible for the ser...,1.0,,,[],0.0,NaN,NaN
1,308322.0,cba15beb4c265876a9828f242b4cf41c,308322,Hauz Khas Social,https://www.zomato.com/HauzKhasSocial?utm_sour...,"9-A & 12, Hauz Khas Village, New Delhi",Hauz Khas Village,New Delhi,1.0,28.5542935327,77.1944381297,,1.0,"Hauz Khas Village, New Delhi",0.0,"Continental, American, Asian, North Indian, Ch...",1300.0,3.0,Rs.,[],"[{'event': {'event_id': 332812, 'friendly_star...",0.0,0.0,OTHER,0.0,,,https://b.zmtcdn.com/data/pictures/2/308322/b5...,4.6,Excellent,3F7E00,13627,0.0,https://www.zomato.com/HauzKhasSocial/photos?u...,https://www.zomato.com/HauzKhasSocial/menu?utm...,https://b.zmtcdn.com/data/pictures/2/308322/b5...,1.0,1.0,0.0,True,zomato://restaurant/308322,1.0,1.0,https://www.zomato.com/HauzKhasSocial/events#t...,[],1,https://www.zomato.com/HauzKhasSocial/order?ut...,,https://www.zomato.com/HauzKhasS

In [8]:
restaurant_df['restaurant.zomato_events'][1117]

[{'event': {'event_id': 305850,
   'friendly_start_date': '01 March',
   'friendly_end_date': '30 April',
   'friendly_timing_str': 'Friday, 1st March - Tuesday, 30th April',
   'start_date': '2019-03-01',
   'end_date': '2019-04-30',
   'end_time': '22:00:00',
   'start_time': '17:30:00',
   'is_active': 1,
   'date_added': '2019-01-31 07:00:12',
   'photos': [{'photo': {'url': 'https://b.zmtcdn.com/data/zomato_events/photos/c32/6f3e2292cfd37f13698e650113679c32_1548898348.jpg',
      'thumb_url': 'https://b.zmtcdn.com/data/zomato_events/photos/c32/6f3e2292cfd37f13698e650113679c32_1548898348.jpg?fit=around%7C100%3A100&crop=100%3A100%3B%2A%2C%2A',
      'order': 0,
      'md5sum': '6f3e2292cfd37f13698e650113679c32',
      'id': 412548,
      'photo_id': 412548,
      'uuid': 1726901202,
      'type': 'NORMAL'}}],
   'restaurants': [],
   'is_valid': 1,
   'share_url': 'http://www.zoma.to/r/0',
   'show_share_url': 0,
   'title': 'Smoky Thursdays',
   'description': "We're adding some mo

In [9]:
def main():
    # URL of the JSON data
    url = "https://raw.githubusercontent.com/Papagoat/brain-assessment/main/restaurant_data.json"
    
    # Fetch data from the URL
    restaurant_data = fetch_data_from_url(url)
    
    # Normalize restaurant data
    restaurant_df = normalize_restaurant_data(restaurant_data)

    # Read in Country-Code Excel file into DataFrame
    country_code_df = pd.read_csv("Country-Code.csv")
    
    # Merge restaurant data with country code data
    merged_restaurant_df = merge_with_country_code(restaurant_df, country_code_df)
    
    # Extract required fields
    final_restaurant_df = extract_required_fields(merged_restaurant_df)
    
    # Rename columns
    final_restaurant_df = rename_columns(final_restaurant_df)
    
    # Convert data types
    final_restaurant_df = convert_data_types(final_restaurant_df)
    
    # Input file path to export restaurants.csv file
    export_to_csv(final_restaurant_df, "C:\\Users\\junke\\Desktop\\Important Documents (JunKeat)\\GovTech-Application\\restaurants.csv")

In [10]:
# pd.set_option('display.max_columns', 500)